In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lnc_analysis import*

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_dir = '/Users/maoyabassiouni/Documents/DATA/Projects/OLNI'

In [5]:
common_mean = 18

# Data

select sites with >10 year range and at least 6 pts

In [6]:
data_f = os.path.join(data_dir ,'model_inputs', 'df_model_all_annual.csv')
df_ts_ = open_lnc_data(data_f)

In [7]:
print('study n years', 2016 - 1995 + 1)
print('study CO2 delta', np.round((np.max(df_ts_['CO2']) - np.min(df_ts_['CO2']))* 10 **6))
print('n datapoints:', len(df_ts_.index))
print('n plots:', len(df_ts_.groupby(['Lat', 'Lon', 'sp']).count().index))
for cm, climate in zip([1, 2, 3], ['Boreal', 'Temperate', 'Mediterranean']):
    print(climate, 'plots:', len(df_ts_[df_ts_['climate_zone']==cm].groupby(['Lat', 'Lon', 'sp']).count().index))
print('number of species', len(list(set(df_ts_['sp'].values))))
print('LNC median %d' % np.round(np.median(df_ts_.groupby(['Lat', 'Lon', 'sp']).median()['LNC'])))
print('LNC mean %d' % np.round(np.mean(df_ts_.groupby(['Lat', 'Lon', 'sp']).median()['LNC'])))

study n years 22
study CO2 delta 43.0
n datapoints: 4228
n plots: 409
Boreal plots: 25
Temperate plots: 338
Mediterranean plots: 46
number of species 32
LNC median 16
LNC mean 18


# Trends & models

- standardize all LNC to same mean 18 mg/g (equal to the mean of means grouped by lat, lon, species)
- sen slope LNC vs year
- growing season climate (terraclimate)
- sunlit leaf

### ICP Observed Trends



In [8]:
df_ts_ = standardize_to_common_mean('LNC', df_ts_, common_mean=common_mean)

In [9]:
trend_result_all = trend_summary_df('LNC_stdm', df_ts_)

### Calculate SWB for each growing season

In [10]:
variant = 'gs_LTavg'
df_ts_ = add_swb_model(df_ts_, variant)

In [11]:
variant = 'gs_yi'
df_ts_ = add_swb_model(df_ts_, variant)

### LTavg climate to trend DF

In [12]:
clim_n = ['TC_par_gs_LTavg',  'TC_pet_gs_LTavg', 'TC_pre_gs_LTavg', 
          'TC_tmean_gs_LTavg', 'TC_vpd_gs_LTavg', 'TC_AI_gs_LTavg',
            's_stressB_gs_yi_canopy_swb', 's_epsilon_gs_yi_canopy_swb']
lt_clim = []

for ii, tti in trend_result_all.iterrows():
    dsix = df_ts_[(df_ts_['Lat']==tti['Lat']) 
                 & (df_ts_['Lon']==tti['Lon'])
                 & (df_ts_['sp']==tti['sp'])]
    lt_clim.append([np.mean(dsix[kk]) for kk in clim_n])
lt_clim = zip(*lt_clim)
for vv, kk in zip(lt_clim, clim_n):
    trend_result_all[kk]=vv

### Calculate P-model 

In [13]:
clim_v = 'gs_yi'
co2_v = 'CO2_yi'
stressB = 1
model_tag ='sunlit_ww'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)

In [14]:
clim_v = 'gs_LTavg'
co2_v = 'CO2_yi'
stressB = 1
model_tag ='sunlit_ww'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)


In [15]:
clim_v = 'gs_yi'
co2_v = 'CO2_LTavg'
stressB = 1
model_tag ='sunlit_ww0'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)

### P-model + SWB

In [16]:
clim_v = 'gs_yi'
co2_v = 'CO2_yi'
stressB= df_ts_['s_stressB_%s_canopy_swb' % clim_v]
model_tag ='sunlit_s_stressB'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)

In [17]:
clim_v = 'gs_LTavg'
co2_v = 'CO2_yi'
stressB= df_ts_['s_stressB_%s_canopy_swb' % clim_v]
model_tag ='sunlit_s_stressB'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)

In [18]:
clim_v = 'gs_yi'
co2_v = 'CO2_LTavg'
stressB= df_ts_['s_stressB_%s_canopy_swb' % clim_v]
model_tag ='sunlit_s_stressB0'
variant = [model_tag, clim_v, co2_v, stressB]

trend_result_all, df_ts_ = add_p_model(df_ts_, trend_result_all, variant, common_mean)

### Random Forest

In [19]:
features_list = ['tmp', 'vpd', 'par', 'stressB', 'CO2']
rfi = 1

trend_result_all, df_ts_, kRF_results, rf_vi = add_rf_model(df_ts_, trend_result_all, features_list, 
                                        target_name = 'LNC_stdm', rfi = rfi)

0 [   0    2    3 ... 4224 4225 4226]
Base Model Performance
Average Error: 0.4743 degrees.
Accuracy = 97.02%.
_m1_k0 Model Performance
Average Error: 1.1228 degrees.
Accuracy = 93.12%.
Variable: tmp                  Importance: 0.04
Variable: vpd                  Importance: 0.11
Variable: par                  Importance: 0.0
Variable: stressB              Importance: 0.14
Variable: CO2                  Importance: 0.71
1 [   0    1    3 ... 4225 4226 4227]
Base Model Performance
Average Error: 0.4708 degrees.
Accuracy = 97.29%.
_m1_k1 Model Performance
Average Error: 1.1100 degrees.
Accuracy = 93.58%.
Variable: tmp                  Importance: 0.06
Variable: vpd                  Importance: 0.09
Variable: par                  Importance: 0.11
Variable: stressB              Importance: 0.29
Variable: CO2                  Importance: 0.45
2 [   0    1    2 ... 4225 4226 4227]
Base Model Performance
Average Error: 0.4827 degrees.
Accuracy = 97.05%.
_m1_k2 Model Performance
Average Error

### Other trends

In [20]:
for y_var in ['TC_pdsi_gs_yi', 'TC_tmean_gs_yi', 'TC_vpd_gs_yi', 'TC_par_gs_yi',
              's_stressB_gs_yi_canopy_swb', 
               's_epsilon_gs_yi_canopy_swb',  
             ]:
    #print(y_var)
    trend_result_ = trend_summary_df(y_var, df_ts_)
    for k in trend_result_:
        if k not in ['Lat', 'Lon', 'sp', 'n_years']:
            trend_result_all[k] = trend_result_[k]

In [21]:
trend_result_all.keys()

Index(['Lat', 'Lon', 'sp', 'n_years', 'trend LNC_stdm', 'delta LNC_stdm',
       'deltaC LNC_stdm', 'trend_p LNC_stdm', 'TC_par_gs_LTavg',
       'TC_pet_gs_LTavg', 'TC_pre_gs_LTavg', 'TC_tmean_gs_LTavg',
       'TC_vpd_gs_LTavg', 'TC_AI_gs_LTavg', 's_stressB_gs_yi_canopy_swb',
       's_epsilon_gs_yi_canopy_swb', 'trend vcmax25_gs_yi_sunlit_ww_stdm',
       'delta vcmax25_gs_yi_sunlit_ww_stdm',
       'deltaC vcmax25_gs_yi_sunlit_ww_stdm',
       'trend_p vcmax25_gs_yi_sunlit_ww_stdm', 'trend nue_gs_yi_sunlit_ww',
       'delta nue_gs_yi_sunlit_ww', 'deltaC nue_gs_yi_sunlit_ww',
       'trend_p nue_gs_yi_sunlit_ww', 'trend vcmax25_gs_LTavg_sunlit_ww_stdm',
       'delta vcmax25_gs_LTavg_sunlit_ww_stdm',
       'deltaC vcmax25_gs_LTavg_sunlit_ww_stdm',
       'trend_p vcmax25_gs_LTavg_sunlit_ww_stdm',
       'trend nue_gs_LTavg_sunlit_ww', 'delta nue_gs_LTavg_sunlit_ww',
       'deltaC nue_gs_LTavg_sunlit_ww', 'trend_p nue_gs_LTavg_sunlit_ww',
       'trend vcmax25_gs_yi_sunlit_ww0_std

# Save results

In [22]:
df_ts_.to_csv('../outputs/Results/df_ts_results.csv') 
    
trend_result_all.to_csv('../outputs/Results/trend_result_all.csv') 


np.save('../outputs/Results/rf_vi.npy', rf_vi)
#np.save('../OLNI_out/kRF_results.npy', kRF_results)